# 06 - Extracción de métricas fisiológicas

## Objetivo
Calcular:
- Velocidad de eritrocitos
- Densidad lineal
- Tasa de suministro


In [ ]:
from src.features.compute_metrics import extract_metrics

metrics = extract_metrics(space_time_img)
print(metrics)

# {'velocity': 115.3, 'density': 78.2, 'supply_rate': 44.6}
